<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 6. Object References, Mutability, and Recycling

A name tis not the object; a name is a separate thing.

## Varialbes are not boxes

Python variables are like reference variables in JAVA.

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

It would be better to say a variable is assigned to the object, not the other way around. Object is created before the assignment, after all.

In [2]:
class Gizmo:
  def __init__(self):
    print(f"Gizmo id: {id(self)}")

x = Gizmo()

Gizmo id: 133687668849872


In [3]:
y = Gizmo() * 10

Gizmo id: 133687668845408


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [4]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '_1',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'quit',
 'x']

## Identity, Equality, and Aliases

In [5]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis is charles

True

In [6]:
id(charles), id(lewis)

(133687666433216, 133687666433216)

`lewis` and `charles` are aliases (two variables are bounded to the same objet)

In [7]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [8]:
alex = {'name': "Charles L. Dodgson", 'born': 1832, 'balance': 950}
# __eq__ implementation in the dict class
alex == charles

True

In [9]:
# identity comparision
alex is not charles

True

Choosing between `==` and `is`
 - `==` compares the values of objects
 - `is` compares their identities
  * cannot be overloaded
  * ex. singleton
  * `x is None`

## Relative Immutability of Tuples

In [10]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t1 == t2

True

In [11]:
id(t1[-1])

133687666556352

In [12]:
t1[-1].append(99)

In [13]:
t1

(1, 2, [30, 40, 99])

In [14]:
id(t1[-1])

133687666556352

In [15]:
t1 == t2

False

## Copies are shallow by default

In [26]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

Or, equivalently,

In [ ]:
l2 = l1[:]

In [27]:
l2 == l1

True

In [28]:
l2 is l1

False

In [31]:
id(l1[1]) == id(l2[1])
id(l1[2]) == id(l2[2])

True

In [20]:
l1[1].append(33)

In [21]:
l2

[3, [55, 44, 33], (7, 8, 9)]

In [30]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print('l1:', l1) # [3, [66, 44], (7, 8, 9), 100]
print('l2:', l2) # [3, [66, 44], (7, 8, 9)]

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]


In [32]:
l2[1] += [33, 22] # += changes the list in place
l2[2] += (10, 11) # OH...! += on a tuple creates a new tuple and rebinds the variable l2[2] here
print('l1:', l1) # [3, [66, 44, 33, 22], (7, 8, 9), 100]
print('l2:', l2) # [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]

l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


## How to do deepcopy

In [33]:
class Bus:
  def __init__(self, passengers=None):
    if passengers is None:
      self.passengers = []
    else:
      self.passengers = list(passengers)

  def pick(self, name):
    self.passengers.append(name)

  def drop(self, name):
    self.passengers.remove(name)

In [34]:
import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(133687667265872, 133687667268176, 133687667276768)

In [35]:
bus1.drop('Bill')
bus2.passengers

['Alice', 'Claire', 'David']

In [36]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

In [37]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(133687664859328, 133687664859328, 133687666582592)

### Cyclic References

In [38]:
a = [10, 20]
b = [a, 30]
a.append(b)

In [39]:
a

[10, 20, [[...], 30]]

In [40]:
from copy import deepcopy
c = deepcopy(a)

In [41]:
c

[10, 20, [[...], 30]]

## Function Parameters as Reference

In [42]:
def f(a, b):
  a += b
  return a

In [43]:
x = 1
y = 2
f(x, y)

3

In [44]:
x, y

(1, 2)

In [47]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [48]:
a, b

([1, 2, 3, 4], [3, 4])

In [49]:
t = (10, 20)
u = (30, 40)
f(t, u)

(10, 20, 30, 40)

In [50]:
t, u

((10, 20), (30, 40))